# Vehicle Price Monitor

**AutoScout Scraper & Indice di Appetibilità Auto**

*Vehicle Price Monitor*, un'applicazione Python che utilizza il web scraping per raccogliere e analizzare gli annunci di auto usate su AutoScout24. L'obiettivo è aiutare l'utende a trovare le migliori offerte di veicoli in base a criteri personalizzati e oggettivi, come prezzo, chilometraggio, potenza e molto altro.

Utilizzando questo strumento, potrai visualizzare un elenco delle **10 auto più appetibili** in base a un indice che tiene conto delle caratteristiche tecniche, della distanza, del prezzo e dell'allestimento. Scopri di più nel progetto e inizia subito a fare analisi!


#### 🎯 Funzionalità principali:
- Raccolta dati da AutoScout24 tramite scraping.
- Calcolo della **distanza** e analisi del **prezzo**, **chilometraggio**, **potenza** e **allestimento**.
- Calcolo di un **Indice di Appetibilità** personalizzato.
- Restituzione delle **10 migliori occasioni**.

#### 🧠 Indice di Appetibilità
L'indice misura l'interesse di un'auto considerando: Anno, prezzo, chilometraggio, distanza, allestimento, potenza e cambio automatico.

#### 🛠️ Come usare:
1. Inserisci l'**URL di AutoScout24** con i risultati.
2. Aggiungi il tuo **comune di residenza**.
3. Aggiungi i modelli dell'auto analizzata
4. Personalizza l'**Indice di Appetibilità** con i tuoi pesi preferiti

# 0. Importazioni

In [1]:
!pip install kaleido -q
import kaleido #required
kaleido.__version__ #0.2.1

import plotly
plotly.__version__ #5.5.0

import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.5 MB/s eta 0:00:00


In [2]:
import os
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
import time
import re
import random
import unicodedata

# Per calcolare la distanza con i cap
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

In [3]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [6]:
import statsmodels.api as sm

In [ ]:
from IPython.display import display, HTML

In [21]:
#url_da_analizzare = str(input("Inserisci l'URL da analizzare: "))
#url_da_analizzare = url_da_analizzare + "&page={}"



# 1. Scraping

In [27]:
# Funzione per formattare la località
def process_location(location_text):
    # 1. Rimuovere 'IT-' dalla stringa
    location_text = location_text.replace('IT-', '')

    # 2. Separare la località, la provincia e il CAP
    # Eseguiamo una regex per separare le informazioni
    match = re.match(r'([A-Za-z\s]+)\s-\s([A-Za-z]+),\s*(\d{5})', location_text)

    if match:
        # Se il match è trovato, ricostruiamo la stringa nel formato desiderato: 'Località, Provincia, CAP'
        locality = match.group(1).strip()
        province = match.group(2).strip()
        cap = match.group(3).strip()

        # Ricostruiamo la stringa
        return f"{locality}, {province}, {cap}"
    else:
        # Se non si trova il formato previsto, restituiamo la location originale
        return location_text

In [28]:
# Funzione per lo scraping
def autoscout_scraper(url_template, max_pages = 20):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    # Initialize all_listings here to store data from all pages
    all_listings = []

    for page in range(1, max_pages + 1):
        url = url_template.format(page)  # Assuming url_template has a placeholder for page number
        print(f"Scraping page {page}...")
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching page {page}: {e}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')

        # Assigning the result of find_all to listings
        listings = soup.find_all('article', class_='cldt-summary-full-item')

        # Checking if listings is empty
        if not listings:
            print(f"Nessun annuncio trovato nella pagina {page}. Terminando.")
            break

        for listing in listings:
            title_tag = listing.find('a', class_='ListItem_title__ndA4s')
            title = title_tag.get_text(strip=True) if title_tag else "N/A"
            # Correzione per l'URL, verifica se è relativo e aggiungi il dominio base
            link = title_tag['href'] if title_tag else "N/A"
            base_url = "https://www.autoscout24.it"  # Modifica il dominio base se necessario
            full_link = base_url + link if link != "N/A" else "N/A"
            #price_tag = listing.find('p', class_='Price_price_APlgs')
            price_tag = listing.find('p', {'data-testid': 'regular-price'})
            price = price_tag.get_text(strip=True) if price_tag else "N/A"
            mileage_tag = listing.find('span', {'data-testid': 'VehicleDetails-mileage_road'})
            mileage = mileage_tag.get_text(strip=True) if mileage_tag else "N/A"
            transmission_tag = listing.find('span', {'data-testid': 'VehicleDetails-transmission'})
            transmission = transmission_tag.get_text(strip=True) if transmission_tag else "N/A"
            registration_date_tag = listing.find('span', {'data-testid': 'VehicleDetails-calendar'})
            registration_date = registration_date_tag.get_text(strip=True) if registration_date_tag else "N/A"
            fuel_type_tag = listing.find('span', {'data-testid': 'VehicleDetails-gas_pump'})
            fuel_type = fuel_type_tag.get_text(strip=True) if fuel_type_tag else "N/A"
            power_tag = listing.find('span', {'data-testid': 'VehicleDetails-speedometer'})
            power = power_tag.get_text(strip=True) if power_tag else "N/A"

            # Check for both location tags
            location_tag_private = listing.find('span', class_='SellerInfo_private__THzvQ')
            location_tag_address = listing.find('span', class_='SellerInfo_address__leRMu')

            # Imposta la variabile "Venditore" in base al tag trovato
            if location_tag_private:
                location_text = location_tag_private.get_text(strip=True)
                venditore = "Privato"
            elif location_tag_address:
                location_text = location_tag_address.get_text(strip=True)
                venditore = "Rivenditore"
            else:
                location_text = "N/A"
                venditore = "N/A"  # Se nessun tag viene trovato, imposta come "N/A"

            # Applica la funzione process_location per formattare la località
            localita = process_location(location_text)

            # Aggiungi il nuovo campo 'Venditore' nel dizionario
            all_listings.append({
                'Annuncio': title,
                'Link': full_link,  # Using the corrected Link variable
                'Prezzo': price,
                'Chilometraggio': mileage,
                'Cambio': transmission,
                'Immatricolazione': registration_date,
                'Carburante': fuel_type,
                'CV': power,
                'Località': localita,
                'Venditore': venditore  # Aggiungi il campo Venditore
            })


        # Importing random for the sleep function
        time.sleep(random.uniform(1, 3.5))

    # Creazione del DataFrame dai dati raccolti
    df = pd.DataFrame(all_listings)
    # Pulizia delle colonne 'Price' e 'Mileage'
    df ['Prezzo'] = df ['Prezzo'].str.replace('[^0-9]','', regex=True).str.replace('.','.')
    df ['Chilometraggio'] = df ['Chilometraggio'].str.replace('[^0-9]','', regex=True)
    # 1. Convertiamo la colonna 'Immatricolazione' in interi, prendendo solo l'anno
    df ['Immatricolazione'] = pd.to_numeric(df['Immatricolazione'].str.split('/').str[1], errors='coerce').astype('Int64')

    # 2. Estraiamo il valore numerico tra parentesi nella colonna 'CV' e lo convertiamo in intero
    df['CV'] = df['CV'].str.extract(r'(?:\(?(\d+)\s?CV\)?)', expand=False).astype(float)

    #3. Sostituzione delle stringhe vuote con NaN
    df ['Prezzo'].replace('', pd.NA, inplace=True)
    df ['Chilometraggio'].replace('', pd.NA, inplace=True)
    df ['Immatricolazione'].replace('', pd.NA, inplace=True)
    df ['CV'].replace('', pd.NA, inplace=True)

    # Conversione in float usando pd.to_numeric con errors='coerce'
    df ['Prezzo'] = pd.to_numeric(df ['Prezzo'], errors = 'coerce' )
    df ['Chilometraggio'] = pd.to_numeric(df[ 'Chilometraggio'], errors = 'coerce')

    # Salvataggio dei dati in un file CSV
    csv_path = '/content/classe_A_Autoscout.csv'
    df.to_csv(csv_path, index=False)
    print(f"Dati salvati in {csv_path}")

In [29]:
#url_template = url_da_analizzare
url_template = "https://www.autoscout24.it/lst/mercedes-benz/classe-a-(tutto)/40019-sant'agata-bolognese?atype=C&cy=I&damaged_listing=exclude&desc=0&lat=44.66517&lon=11.135&powertype=kw&search_id=aal2hzkars&sort=standard&source=homepage_search-mask&ustate=N%2CU&zipr=150&page={}"
autoscout_scraper(url_template)

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Dati salvati in /content/classe_A_Autoscout.csv


<ipython-input-28-b4efd2c8415c>:99: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df ['Prezzo'].replace('', pd.NA, inplace=True)
<ipython-input-28-b4efd2c8415c>:100: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [30]:
data_autoscout = pd.read_csv('/content/classe_A_Autoscout.csv')
data_autoscout.head(5)

,Annuncio,Link,Prezzo,Chilometraggio,Cambio,Immatricolazione,Carburante,CV,Località,Venditore
0,Mercedes-BenzA 160BlueEFFICIENCY EleganceAcqui...,https://www.autoscout24.it/annunci/mercedes-be...,2500.0,174800,Manuale,2011.0,Benzina,95.0,"Italo, Stefano, Mirko, Rossano, Francesco, Mar...",Rivenditore
1,Mercedes-BenzA 180A 180 d Automatic SportFari ...,https://www.autoscout24.it/annunci/mercedes-be...,19900.0,88669,Semiautomatico,2019.0,Diesel,116.0,Castrenze Tamburello • 40043 Marzabotto - Bologna,Rivenditore
2,Mercedes-BenzA 180A 180 d Automatic PremiumSed...,https://www.autoscout24.it/annunci/mercedes-be...,21900.0,25800,Automatico,2018.0,Diesel,116.0,Usato Selezionato Gruppo Morini • 40068 San La...,Rivenditore
3,Mercedes-BenzA 160Business A 160POSSIBILITA' D...,https://www.autoscout24.it/annunci/mercedes-be...,18500.0,61500,Manuale,2021.0,Benzina,109.0,Tuacar PC- PR • 29010 San Nicolò - Pc,Rivenditore
4,Mercedes-BenzA 160A 160 be Special editionSpec...,https://www.autoscout24.it/annunci/mercedes-be...,4000.0,100000,Manuale,2010.0,Benzina,95.0,Antonio • 31040 Trevignano - Treviso - TV,Rivenditore


In [31]:
n_unici = data_autoscout['Link'].nunique()
print(f"Numero di link unici: {n_unici}\n")

data_autoscout.info()

Numero di link unici: 380

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Annuncio          381 non-null    object 
 1   Link              381 non-null    object 
 2   Prezzo            374 non-null    float64
 3   Chilometraggio    381 non-null    int64  
 4   Cambio            381 non-null    object 
 5   Immatricolazione  374 non-null    float64
 6   Carburante        381 non-null    object 
 7   CV                381 non-null    float64
 8   Località          381 non-null    object 
 9   Venditore         381 non-null    object 
dtypes: float64(3), int64(1), object(6)
memory usage: 29.9+ KB


# 2. Preparazione dataset

In [32]:
data_autoscout = pd.read_csv('/content/classe_A_Autoscout.csv')
data_autoscout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Annuncio          381 non-null    object 
 1   Link              381 non-null    object 
 2   Prezzo            374 non-null    float64
 3   Chilometraggio    381 non-null    int64  
 4   Cambio            381 non-null    object 
 5   Immatricolazione  374 non-null    float64
 6   Carburante        381 non-null    object 
 7   CV                381 non-null    float64
 8   Località          381 non-null    object 
 9   Venditore         381 non-null    object 
dtypes: float64(3), int64(1), object(6)
memory usage: 29.9+ KB


In [33]:
val_doppi = data_autoscout['Link'].duplicated()
print(f"Ci sono dei valori doppi? {val_doppi.any()}")
if val_doppi.any() == True:
    print(f"Se si quanti ce ne sono? {val_doppi.sum()}, su un totale di {len(data_autoscout)}")

Ci sono dei valori doppi? True
Se si quanti ce ne sono? 1, su un totale di 381


In [34]:
data = data_autoscout.copy()

## 2.1. Calcolo Distanza

Import Dataset con i cap

In [35]:
raw = 'https://raw.githubusercontent.com/'

In [36]:
url_cap = raw + 'FedeGambe/Vehicle_Price_Monitor/main/Materiali/Data/gi_comuni_cap.csv'
cap= pd.read_csv(url_cap, delimiter=';')

only_cap = cap[['cap', 'denominazione_ita', 'sigla_provincia']]
only_cap = only_cap.rename(columns={'denominazione_ita': 'Comune', 'cap': 'CAP', 'sigla_provincia': 'Provincia'})

def normalizza_testo(testo): # Funzione per rimuovere gli accenti e convertire in minuscolo
    if pd.isnull(testo):
        return testo
    # Normalizza Unicode per rimuovere accenti
    testo = unicodedata.normalize('NFKD', testo)
    testo = ''.join([c for c in testo if not unicodedata.combining(c)])
    # Sostituisci apostrofi con spazio, se non c'è già uno spazio accanto
    testo = re.sub(r"(?<! )'(?! )", ' ', testo)
    # Converte tutto in minuscolo
    return testo.lower()

only_cap['Comune'] = only_cap['Comune'].apply(normalizza_testo)
only_cap['Provincia'] = only_cap['Provincia'].apply(normalizza_testo)

print("Cap di tutta italia\n",only_cap.info(), "\n")

#Eliminare i duplicati dei cap delle grandi città
only_cap_unique = only_cap.sort_values('Comune').drop_duplicates(subset='CAP', keep='first')
print("Cap duplicati eliminati delle grandi città\n",only_cap_unique.info(), "\n")
only_cap_unique.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8490 entries, 0 to 8489
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CAP        8490 non-null   int64 
 1   Comune     8489 non-null   object
 2   Provincia  8374 non-null   object
dtypes: int64(1), object(2)
memory usage: 199.1+ KB
Cap di tutta italia
 None 

<class 'pandas.core.frame.DataFrame'>
Index: 4657 entries, 3519 to 7937
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CAP        4657 non-null   int64 
 1   Comune     4657 non-null   object
 2   Provincia  4569 non-null   object
dtypes: int64(1), object(2)
memory usage: 145.5+ KB
Cap duplicati eliminati delle grandi città
 None 



,CAP,Comune,Provincia
3519,35031,abano terme,pd
8026,26834,abbadia cerreto,lo
7942,23821,abbadia lariana,lc
4658,53021,abbadia san salvatore,si
7781,9071,abbasanta,or


In [37]:
only_cap[only_cap['CAP'] == 50100]

,CAP,Comune,Provincia


In [38]:
def pulisci_indirizzo(indirizzo):
    # Separiamo il nome dal resto dell'indirizzo
    parti = indirizzo.split("•")

    if len(parti) > 1:
        indirizzo = parti[1].strip()  # Prendiamo solo la parte dell'indirizzo

    # Cerchiamo di estrarre il CAP (che dovrebbe essere un numero a 5 cifre)
    cap = re.search(r'\b\d{5}\b', indirizzo)

    # Estraiamo la città e la provincia (se presente)
    if cap:
        cap = cap.group(0)  # Prendiamo il CAP trovato
        indirizzo = indirizzo.replace(cap, "").strip()  # Rimuoviamo il CAP dalla stringa

        # Separiamo città e provincia (se presenti)
        parti_indirizzo = indirizzo.split("-")

        if len(parti_indirizzo) == 2:  # Caso con provincia
            citta = parti_indirizzo[0].strip()
            provincia = parti_indirizzo[1].strip()
        elif len(parti_indirizzo) == 1:  # Caso senza provincia
            citta = parti_indirizzo[0].strip()
            provincia = None
        else:
            citta = None
            provincia = None

        return cap, citta, provincia
    else:
        return None, None, None  # Se non c'è un CAP, restituiamo None

# Applicare la funzione alla colonna 'Località'
data[['CAP', 'Città', 'Provincia']] = data['Località'].apply(lambda x: pd.Series(pulisci_indirizzo(x)))
# Creare la nuova colonna 'Località' concatenando solo i valori non None
data['Località'] = data.apply(lambda row: ', '.join(filter(None, [row['CAP'], row['Città'], row['Provincia']])), axis=1)
# Eliminare la colonna 'Provincia'
data = data.drop(columns=['Provincia'])
data['CAP'] = data['CAP'].astype('Int64')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Annuncio          381 non-null    object 
 1   Link              381 non-null    object 
 2   Prezzo            374 non-null    float64
 3   Chilometraggio    381 non-null    int64  
 4   Cambio            381 non-null    object 
 5   Immatricolazione  374 non-null    float64
 6   Carburante        381 non-null    object 
 7   CV                381 non-null    float64
 8   Località          381 non-null    object 
 9   Venditore         381 non-null    object 
 10  CAP               381 non-null    Int64  
 11  Città             296 non-null    object 
dtypes: Int64(1), float64(3), int64(1), object(7)
memory usage: 36.2+ KB


In [39]:
data = data.merge(only_cap_unique[['CAP', 'Comune']], on='CAP', how='left')
data = data.drop(columns=['Località', 'Città'])
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 356 entries, 0 to 380
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Annuncio          356 non-null    object 
 1   Link              356 non-null    object 
 2   Prezzo            356 non-null    float64
 3   Chilometraggio    356 non-null    int64  
 4   Cambio            356 non-null    object 
 5   Immatricolazione  356 non-null    float64
 6   Carburante        356 non-null    object 
 7   CV                356 non-null    float64
 8   Venditore         356 non-null    object 
 9   CAP               356 non-null    Int64  
 10  Comune            356 non-null    object 
dtypes: Int64(1), float64(3), int64(1), object(6)
memory usage: 33.7+ KB


Calcolo Distanza

In [40]:
url_distanza = raw + 'FedeGambe/Vehicle_Price_Monitor/main/Materiali/Data/italy_geo.json'
distanza = pd.read_json(url_distanza)
distanza = distanza.rename(columns={'comune': 'Comune', 'lng': 'Longitudine', 'lat': 'Latitudine'})
distanza['Comune'] = distanza['Comune'].apply(normalizza_testo)
distanza.drop(columns=['istat'], inplace=True)
distanza = distanza.drop([7978, 7979])
distanza.head(5)

,Comune,Longitudine,Latitudine
0,aglie,7.7686,45.363433
1,airasca,7.48443104,44.916886
2,ala di stura,7.30434392,45.315110
3,albiano d ivrea,7.94914491,45.433893
4,alice superiore,7.77701858,45.460094


In [41]:
#comune_riferimento = str(input("Inserisci il tuo comune di residenza")) # Filtra il comune di riferimento
comune_riferimento = "Sant'Agata Bolognese" # Filtra il comune di riferimento
comune_riferimento = normalizza_testo(comune_riferimento)
comune_rif = distanza[distanza['Comune'] == comune_riferimento]

# Estrazione latitudine e longitudine per il comune di riferimento
lat_rif = comune_rif['Latitudine'].values[0]
lon_rif = comune_rif['Longitudine'].values[0]

# Funzione per calcolare la distanza tra il comune di riferimento e gli altri comuni
def calcola_distanza(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).kilometers

# Calcolare la distanza per tutti gli altri comuni
distanza['Distanza'] = distanza.apply(
    lambda row: calcola_distanza(
        lat_rif,
        lon_rif,
        row['Latitudine'],
        row['Longitudine'])
    , axis=1)

distanza_sorted = distanza.sort_values(by='Distanza')
distanza_sorted[['Comune', 'Distanza']].head(10)

,Comune,Distanza
3941,sant agata bolognese,0.000000
3938,san giovanni in persiceto,5.151917
3912,crevalcore,6.540985
3878,ravarino,7.178344
3871,nonantola,7.340526
3850,castelfranco emilia,9.926182
3846,bomporto,10.040101
3935,sala bolognese,11.987551
3845,bastiglia,12.654440
3907,castello d argile,13.040810


Unione distanze con il Dataframe `Data`

In [42]:
data = data.merge(distanza[['Comune', 'Distanza']], on='Comune', how='left')
data['Distanza'] = data['Distanza'].round(2)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356 entries, 0 to 355
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Annuncio          356 non-null    object 
 1   Link              356 non-null    object 
 2   Prezzo            356 non-null    float64
 3   Chilometraggio    356 non-null    int64  
 4   Cambio            356 non-null    object 
 5   Immatricolazione  356 non-null    float64
 6   Carburante        356 non-null    object 
 7   CV                356 non-null    float64
 8   Venditore         356 non-null    object 
 9   CAP               356 non-null    Int64  
 10  Comune            356 non-null    object 
 11  Distanza          355 non-null    float64
dtypes: Int64(1), float64(4), int64(1), object(6)
memory usage: 33.9+ KB


In [43]:
print("Null value di Distanza:",data['Distanza'].isna().sum())
print("Null value di Città:",data['Comune'].isna().sum())
print("Null value di CAP:",data['CAP'].isna().sum())

Null value di Distanza: 1
Null value di Città: 0
Null value di CAP: 0


In [44]:
data = data.dropna()

## 2.2. Formattazione

In [45]:
#per uniformare i km e togliere in alcuni casi la parola km
data['Chilometraggio'] = data['Chilometraggio'].apply(
    lambda x: re.sub(r'\s*[kK][mM]', '', x) if isinstance(x, str) else x)
data['Chilometraggio'] = pd.to_numeric(data['Chilometraggio'], errors='coerce').astype('Int64')

In [46]:
#Funzione per pulire e formattare il prezzo
def clean_price(price):
    if isinstance(price, (int, float)):     # Se il prezzo è numerico, arrotondalo a 2 cifre decimali
        return round(price, 2)
        # Se il prezzo è una stringa, rimuovi i caratteri non numerici
        # e formattalo come numero in virgola mobile
    try:
        price = re.sub(r'[^\d,]', '', price)
        price = price.replace(',', '.')
        return round(float(price), 2)
    except (ValueError, TypeError):
        return pd.NA  # Restituisci NaN per valori non validi

data['Prezzo'] = data['Prezzo'].apply(clean_price) # Applica la funzione alla colonna 'Prezzo'

In [47]:
def cambio_score(cambio):
    if isinstance(cambio, str):
        cambio_lower = cambio.lower()
        if 'auto' in cambio_lower:
            return 1
        elif 'manual' in cambio_lower:
            return 0
    return 0  # Default per valori non riconosciuti o nulli

data['Cambio'] = data['Cambio'].apply(cambio_score)
print("I nuovi valori unici della feature Cambio sono:", data['Cambio'].unique())

I nuovi valori unici della feature Cambio sono: [0 1]


In [80]:
data['Indice_Appetibilità'] = data['Indice_Appetibilità'].round(2)
data['Prezzo'] = data['Prezzo'].round(2)
data['CV'] = data['CV'].round(2)
data['Distanza'] = data['Distanza'].round(2)

Calcolo Anni

In [48]:
data['Immatricolazione'] = pd.to_numeric(data['Immatricolazione'], errors='coerce').astype('Int64')
data['Anni'] = data['Immatricolazione'].apply(lambda x: datetime.now().year - x if pd.notna(x) else None).astype('Int64')
#data['Anni'] = data.apply(lambda row: datetime.now().year - row['Immatricolazione_anno'] if pd.isna(row['Anni']) and pd.notna(row['Immatricolazione_anno']) else row['Anni'], axis=1)

In [49]:
data['Venditore'] = data['Venditore'].map({'Rivenditore': 1, 'Privato': 0})

## 2.3 Indice di Appetibilità

In [50]:
#Determinare l'allestimento dal titolo dell'annuncio
keywords = ["Sport", "Business", "AMG", "AMG line", "amg-line", "Elegance", "Premium", "Advanced", "Progressive", "Executive"]
# Funzione per determinare l'allestimento
def get_allestimento(annuncio):
    annuncio = annuncio.lower()  # Rende il testo case insensitive
    for keyword in keywords:
        if keyword.lower() in annuncio:
            # Se troviamo "AMG line", "amg-line" o "Premium", li sostituiamo con "Premium AMG line"
            if 'amg line' in annuncio or 'amg-line' in annuncio or 'premium' in annuncio:
                return 'Premium AMG line'
            return keyword
    return None  # Restituisce None se nessun valore corrisponde

# Aggiungi la nuova colonna 'Allestimento' applicando la funzione a ogni riga
data['Allestimento'] = data['Annuncio'].apply(get_allestimento)
print(pd.unique(data['Allestimento']))

['Elegance' 'Sport' 'Premium AMG line' 'Business' None 'Executive' 'AMG'
 'Advanced']


In [51]:
def normalizzata_inv(value, min_value, max_value):
    return (max_value - value) / (max_value - min_value)

def normalizzata(value, min_value, max_value):
    return (value - min_value) / (max_value - min_value)

def allestimento_score(allestimento):
    if allestimento == 'Premium AMG line' or allestimento == 'AMG' :
        return 1
    elif allestimento in ['Sport', 'Advanced', 'Elegance']:
        return 0.4
    elif allestimento in ['Business', 'Executive']:
        return 0.2
    else:
        return 0

anni_norm = data['Anni'].apply(lambda x: normalizzata_inv(x, data['Anni'].min(), data['Anni'].max()))
prezzo_norm = data['Prezzo'].apply(lambda x: normalizzata_inv(x, data['Prezzo'].min(), data['Prezzo'].max()))
chilometraggio_norm = data['Chilometraggio'].apply(lambda x: normalizzata_inv(x, data['Chilometraggio'].min(), data['Chilometraggio'].max()))
distanza_norm = data['Distanza'].apply(lambda x: normalizzata_inv(x, data['Distanza'].min(), data['Distanza'].max()))
cv_norm = data['CV'].apply(lambda x: normalizzata(x, data['CV'].min(), data['CV'].max()))
allestimento_score_col = data['Allestimento'].apply(allestimento_score)

# Calcolo dell'indice appetibilità
data['Indice_Appetibilità'] = (
    0.15 * anni_norm +              #Anni bassi è migliore
    0.25 * prezzo_norm +            #Prezzo basso è migliore
    0.20 * chilometraggio_norm +    #Chilometraggio basso è migliore
    0.10 * distanza_norm +          #Distanza bassa è migliore
    0.30 * allestimento_score_col + #Allestimento
    0.10 * cv_norm +                #CV alto è migliore
    0.50 * data['Cambio']           #Cambio automatico è migliore
)

## 2.4. Sistemazione e controllo finale

In [52]:
data = data[['Annuncio', 'Link', 'Indice_Appetibilità', 'Prezzo', 'Anni' ,'Immatricolazione', 'Chilometraggio', 'Cambio',
             'Carburante', 'CV', 'Allestimento','Venditore', 'Distanza','Comune', 'CAP']]

In [82]:
data.head(5)

,Annuncio,Link,Indice_Appetibilità,Prezzo,Anni,Immatricolazione,Chilometraggio,Cambio,Carburante,CV,Allestimento,Venditore,Distanza,Comune,CAP
0,Mercedes-BenzA 160BlueEFFICIENCY EleganceAcqui...,https://www.autoscout24.it/annunci/mercedes-be...,0.61,2500.0,14,2011,174800,0,Benzina,95.0,Elegance,1,14.77,pieve di cento,40066
1,Mercedes-BenzA 180A 180 d Automatic SportFari ...,https://www.autoscout24.it/annunci/mercedes-be...,1.12,19900.0,6,2019,88669,1,Diesel,116.0,Sport,1,36.47,marzabotto,40043
2,Mercedes-BenzA 180A 180 d Automatic PremiumSed...,https://www.autoscout24.it/annunci/mercedes-be...,1.33,21900.0,7,2018,25800,1,Diesel,116.0,Premium AMG line,1,30.65,san lazzaro di savena,40068
3,Mercedes-BenzA 160Business A 160POSSIBILITA' D...,https://www.autoscout24.it/annunci/mercedes-be...,0.54,18500.0,4,2021,61500,0,Benzina,109.0,Business,1,131.42,agazzano,29010
4,Mercedes-BenzA 160A 160 be Special editionSpec...,https://www.autoscout24.it/annunci/mercedes-be...,0.44,4000.0,15,2010,100000,0,Benzina,95.0,None,1,164.61,cessalto,31040


In [54]:
duplicati = data['Link'].duplicated().sum()
print(f"Numero di righe duplicate: {duplicati}")
data_data_red = data.drop_duplicates(subset='Link')

n_unici = data['Link'].nunique()
print(f"Numero di link unici: {n_unici}")

Numero di righe duplicate: 1
Numero di link unici: 354


In [55]:
# data sorted by Indice_Appetibilità
data_sorted_indx = data.sort_values(by='Indice_Appetibilità', ascending=False)
print(data_sorted_indx.head(5))

                                              Annuncio  \
175                     Mercedes-BenzA 200Premium auto   
2    Mercedes-BenzA 180A 180 d Automatic PremiumSed...   
205  Mercedes-BenzA 180A 180 d Automatic Premium AM...   
311  Mercedes-BenzA 180Premium AMG VARI COLORI E AL...   
218        Mercedes-BenzA 180A 180 d Automatic Premium   

                                                  Link  Indice_Appetibilità  \
175  https://www.autoscout24.it/annunci/mercedes-be...             1.344886   
2    https://www.autoscout24.it/annunci/mercedes-be...             1.329523   
205  https://www.autoscout24.it/annunci/mercedes-be...             1.320430   
311  https://www.autoscout24.it/annunci/mercedes-be...             1.311292   
218  https://www.autoscout24.it/annunci/mercedes-be...             1.309805   

      Prezzo  Anni  Immatricolazione  Chilometraggio  Cambio Carburante  \
175  27800.0     1              2024           38000       1    Benzina   
2    21900.0     7        

## 2.5. Dataset dummy

In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 355 entries, 0 to 355
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Annuncio             355 non-null    object 
 1   Link                 355 non-null    object 
 2   Indice_Appetibilità  355 non-null    float64
 3   Prezzo               355 non-null    float64
 4   Anni                 355 non-null    Int64  
 5   Immatricolazione     355 non-null    Int64  
 6   Chilometraggio       355 non-null    Int64  
 7   Cambio               355 non-null    int64  
 8   Carburante           355 non-null    object 
 9   CV                   355 non-null    float64
 10  Allestimento         308 non-null    object 
 11  Venditore            355 non-null    int64  
 12  Distanza             355 non-null    float64
 13  Comune               355 non-null    object 
 14  CAP                  355 non-null    Int64  
dtypes: Int64(4), float64(4), int64(2), object(5)


In [57]:
data_dummy = data.copy()
data_dummy= data_dummy.drop (columns=['Annuncio', 'Link', 'Comune','Immatricolazione', 'CAP'])

In [58]:
data_dummy.select_dtypes(include=['object']).info()

<class 'pandas.core.frame.DataFrame'>
Index: 355 entries, 0 to 355
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Carburante    355 non-null    object
 1   Allestimento  308 non-null    object
dtypes: object(2)
memory usage: 8.3+ KB


In [59]:
data_dummy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 355 entries, 0 to 355
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Indice_Appetibilità  355 non-null    float64
 1   Prezzo               355 non-null    float64
 2   Anni                 355 non-null    Int64  
 3   Chilometraggio       355 non-null    Int64  
 4   Cambio               355 non-null    int64  
 5   Carburante           355 non-null    object 
 6   CV                   355 non-null    float64
 7   Allestimento         308 non-null    object 
 8   Venditore            355 non-null    int64  
 9   Distanza             355 non-null    float64
dtypes: Int64(2), float64(4), int64(2), object(2)
memory usage: 31.2+ KB


###Carburante

In [60]:
print(pd.unique(data['Carburante']))
#['Diesel' 'Benzina' 'Ibrida' 'Sequenziale' 'Elettrica/Benzina']

['Benzina' 'Diesel' 'Elettrica/Benzina' 'GPL']


In [61]:
data_dummy['Carburante'] = (data_dummy['Carburante'].fillna('Unknown').str.strip().str.lower())
data_dummy['Carburante'] = data_dummy['Carburante'].replace('elettrica/benzina', 'ibrida')

carburanti = ['diesel', 'benzina', 'ibrida', 'gpl']
for carburante in carburanti:
    data_dummy[f'is_{carburante}'] = data_dummy['Carburante'].apply(lambda x: 1 if x == carburante else 0)

data_dummy[['Carburante', 'is_diesel', 'is_benzina', 'is_ibrida', 'is_gpl']].head()

,Carburante,is_diesel,is_benzina,is_ibrida,is_gpl
0,benzina,0,1,0,0
1,diesel,1,0,0,0
2,diesel,1,0,0,0
3,benzina,0,1,0,0
4,benzina,0,1,0,0


In [62]:
data_dummy.drop(columns=['Carburante'], inplace=True)

###Allestimento

In [63]:
print(pd.unique(data_dummy['Allestimento']))
#['Sport' 'Premium AMG line' 'Business' None 'Executive' 'Advanced']

['Elegance' 'Sport' 'Premium AMG line' 'Business' None 'Executive' 'AMG'
 'Advanced']


In [64]:
data_dummy['Allestimento'] = (
    data_dummy['Allestimento']
    .fillna('Unknown')                      # Gestisce eventuali NaN
    .str.strip()                            # Rimuove spazi extra
    .str.lower()                            # Porta tutto in minuscolo
)

premium_amg_line = ['premium amg line']
amg = ['amg']
middle = ['sport', 'advanced', 'elegance']
base = ['business', 'executive']

data_dummy['is_premium_amg_line'] = data_dummy['Allestimento'].isin(premium_amg_line).astype(int)
data_dummy['is_amg'] = data_dummy['Allestimento'].isin(amg).astype(int)
data_dummy['is_middle'] = data_dummy['Allestimento'].isin(middle).astype(int)
data_dummy['is_base'] = data_dummy['Allestimento'].isin(base).astype(int)

data_dummy[['Allestimento', 'is_premium_amg_line', 'is_amg', 'is_middle', 'is_base']].head()

,Allestimento,is_premium_amg_line,is_amg,is_middle,is_base
0,elegance,0,0,1,0
1,sport,0,0,1,0
2,premium amg line,1,0,0,0
3,business,0,0,0,1
4,unknown,0,0,0,0


In [65]:
data_dummy.drop(columns=['Allestimento'], inplace=True)

# 3. Comprendere il Prezzo

In [66]:
X = data_dummy.drop(columns=['Prezzo', 'Indice_Appetibilità'])
y = data_dummy['Prezzo']

In [67]:
corr_matrix = data_dummy.corr()
target_corr = corr_matrix['Prezzo'].drop('Prezzo')
target_corr_sorted = target_corr.reindex(target_corr.abs().sort_values(ascending=False).index)

corr_df = pd.DataFrame({
    'Variabile': target_corr_sorted.index,
    'Correlazione': target_corr_sorted.values})


fig_cor_y = px.bar(corr_df,x='Correlazione',y='Variabile',
    orientation='h',
    color='Correlazione', color_continuous_scale='RdBu')

fig_cor_y.update_traces(hovertemplate='<b>%{y}</b><br>Correlazione: %{x:.2f}')
fig_cor_y.update_layout(title={'text': "Correlazione delle feature con il Prezzo",'x': 0.5,
                         'xanchor': 'center', 'font': {'size': 20, 'family': 'Arial', 'weight': 'bold'},},
                  yaxis=dict(autorange="reversed"),  height=600, width=800, )
fig_cor_y.show()

In [68]:
correlation_matrix = X.corr()
original_columns = list(correlation_matrix.columns)

fig_corr_X = ff.create_annotated_heatmap(z=correlation_matrix.values, x=original_columns, y=original_columns,
    annotation_text=np.around(correlation_matrix.values, decimals=2),showscale=True, colorscale='RdBu',reversescale=True,hoverinfo='text',)

fig_corr_X.update_layout(
    title={'text': "Correlation Matrix tra le features",'x': 0.5,'xanchor': 'center',  'font': {'size': 20, 'family': 'Arial', 'weight': 'bold'},},
    margin=dict(t=200, b=25, l=0, r=0),height=600,width=1000)
fig_corr_X.show()

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [70]:
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)


y_pred = reg.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 5534048.671970214
R-squared: 0.9427183400203881


In [71]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 355 entries, 0 to 355
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Anni                 355 non-null    Int64  
 1   Chilometraggio       355 non-null    Int64  
 2   Cambio               355 non-null    int64  
 3   CV                   355 non-null    float64
 4   Venditore            355 non-null    int64  
 5   Distanza             355 non-null    float64
 6   is_diesel            355 non-null    int64  
 7   is_benzina           355 non-null    int64  
 8   is_ibrida            355 non-null    int64  
 9   is_gpl               355 non-null    int64  
 10  is_premium_amg_line  355 non-null    int64  
 11  is_amg               355 non-null    int64  
 12  is_middle            355 non-null    int64  
 13  is_base              355 non-null    int64  
dtypes: Int64(2), float64(2), int64(10)
memory usage: 42.3 KB


In [72]:
#@title Confronto Variabili: Regressione Polinomiale
grado = 3
titoli = list(X.columns)
# Calculate the number of rows and columns needed for subplots
n_cols = 3  # Number of columns
n_rows = (len(titoli) + n_cols - 1) // n_cols  # Calculate rows needed

fig = make_subplots(rows=n_rows, cols=n_cols, subplot_titles=titoli)

for i in range(len(titoli)): # Loop through the number of features
    x = X.iloc[:, i].values.reshape(-1, 1)
    # Convert x to a NumPy array before calling flatten()
    x = np.array(x)
    y_vals = y.values

    # -------------------------------
    # 🔸 Regressione Polinomiale
    grado = 3
    poly = PolynomialFeatures(degree=grado)
    x_poly = poly.fit_transform(x)
    model = LinearRegression().fit(x_poly, y_vals)

    x_range = np.linspace(x.min(), x.max(), 300).reshape(-1, 1)
    x_range_poly = poly.transform(x_range)
    y_pred_poly = model.predict(x_range_poly)

    # -------------------------------
    # Tracce
    row = i // n_cols + 1 # Calculate the correct row for the subplot
    col = i % n_cols + 1  # Calculate the correct column for the subplot

    # Dati ,marker=dict( opacity=0.6)
    fig.add_trace(go.Scatter(x=x.flatten(), y=y_vals,mode='markers', marker=dict(size=3, color='deepskyblue', opacity=0.3),name='Dati',showlegend = False),row=row, col=col) #per legenda mettere: showlegend=(i == 0)
    # Polinomiale
    fig.add_trace(go.Scatter(x=x_range.flatten(), y=y_pred_poly,mode='lines',line=dict(color='orange', width=2),name=f'Polinomiale di grado: {grado}',showlegend = False),row=row, col=col) #per legenda mettere: showlegend=(i == 0)

    fig.update_yaxes(title_text="Prezzo", row=row, col=col, title_font=dict(size=10), tickfont=dict(size=9))

fig.update_layout(
    title={'text': f"Confronto: Regressione Polinomiale (grado {grado})", 'x': 0.5, 'xanchor': 'center', 'font': {'size': 20, 'family': 'Arial', 'weight': 'bold'}},
    template='plotly_dark',height=1200,width=1500, )

fig.update_annotations(font_size=10)
fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=True)

fig.show()

# Conclusione

Le auto migliori secondo l'Indice di Appettibilità sono:

In [ ]:
#Annotazione

#Venditore: 0 = Privato, 1 = Rivenditore
#Cambio: 0 = Manuale, 1 = Automatico/Semiauto

In [96]:
#@title Le 10 auto migliori secondo l'Indice di Appettibilità
top_ten = data.nlargest(10, 'Indice_Appetibilità')
for index, row in top_ten_appetibility.iterrows():
    link = row['Link']
    display(HTML(f"Numero indice: {index} -> <a href='{link}' target='_blank'>{link}</a>"))
print()
top_ten

,Annuncio,Link,Indice_Appetibilità,Prezzo,Anni,Immatricolazione,Chilometraggio,Cambio,Carburante,CV,Allestimento,Venditore,Distanza,Comune,CAP
175,Mercedes-BenzA 200Premium auto,https://www.autoscout24.it/annunci/mercedes-be...,1.34,27800.0,1,2024,38000,1,Benzina,163.0,Premium AMG line,0,25.11,bologna,40141
2,Mercedes-BenzA 180A 180 d Automatic PremiumSed...,https://www.autoscout24.it/annunci/mercedes-be...,1.33,21900.0,7,2018,25800,1,Diesel,116.0,Premium AMG line,1,30.65,san lazzaro di savena,40068
205,Mercedes-BenzA 180A 180 d Automatic Premium AM...,https://www.autoscout24.it/annunci/mercedes-be...,1.32,31900.0,1,2024,24024,1,Diesel,116.0,Premium AMG line,1,23.55,casalecchio di reno,40033
41,Mercedes-BenzA 200d Automatic PremiumCONCESSIO...,https://www.autoscout24.it/annunci/mercedes-be...,1.31,31900.0,3,2022,34508,1,Diesel,150.0,Premium AMG line,1,25.11,bologna,40127
105,Mercedes-BenzA 35 AMG4Matic-1000€ solo domenic...,https://www.autoscout24.it/annunci/mercedes-be...,1.31,32900.0,5,2020,72500,1,Benzina,306.0,AMG,1,31.73,castenaso,40055
200,Mercedes-BenzA 180d Premium Night edition auto,https://www.autoscout24.it/annunci/mercedes-be...,1.31,26900.0,4,2021,44549,1,Diesel,116.0,Premium AMG line,0,31.29,sasso marconi,40037
218,Mercedes-BenzA 180A 180 d Automatic Premium,https://www.autoscout24.it/annunci/mercedes-be...,1.31,27400.0,4,2021,46312,1,Diesel,116.0,Premium AMG line,1,23.55,casalecchio di reno,40033
268,Mercedes-BenzA 180PREMIUM AMG PACK LUCI VARI C...,https://www.autoscout24.it/annunci/mercedes-be...,1.31,30500.0,2,2023,19000,1,Benzina,163.0,Premium AMG line,1,77.97,arcole,37040
311,Mercedes-BenzA 180Premium AMG VARI COLORI E AL...,https://www.autoscout24.it/annunci/mercedes-be...,1.31,29900.0,1,2024,15500,1,Benzina,136.0,Premium AMG line,1,77.97,arcole,37040
81,"Mercedes-BenzA 35 AMG4MaticCerchi in lega, Tra...",https://www.autoscout24.it/annunci/mercedes-be...,1.30,32900.0,5,2020,72500,1,Benzina,306.0,AMG,1,42.86,ferrara,44122
